In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import h5py
import seaborn as sns
from scipy.special import logit
from scipy.special import expit
from scipy.interpolate import RegularGridInterpolator
from scipy.stats import entropy
from pylab import rcParams
import sys
import torch
import scipy
import corner
import copy

import sys
sys.path.append('../')
from populations.bbh_models import read_hdf5
import populations.bbh_models as read_models
from populations.Flowsclass_dev import FlowModel
from populations.utils.bounded_Nd_kde import Bounded_Nd_kde
from populations import gw_obs

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


glasflow is using its own internal version of nflows


In [2]:
param = ['mchirp','q', 'chieff', 'z']
no_params = len(param)
channel_label = 'CE'
chi_b = [0.0,0.1,0.2,0.5]
alpha_CE = [0.2,0.5,1.0,2.,5.]

channel_ids = {'CE':0, 'CHE':1,'GC':2,'NSC':3, 'SMT':4}
channel_id = channel_ids[channel_label] #will be 0, 1, 2, 3, or 4
channel_samples = [4e6,864124,896611,582961, 4e6]
no_binaries = int(channel_samples[channel_id])

models_path ='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
popsynth_outputs = read_hdf5(models_path, channel_label) # read all data from hdf5 file

models_dict = dict.fromkeys(popsynth_outputs.keys())
weights_dict = dict.fromkeys(popsynth_outputs.keys())


for key in popsynth_outputs.keys():
    models_dict[key] = popsynth_outputs[key][param]
    weights_dict[key]= popsynth_outputs[key]['weight']

In [3]:
params = ['mchirp','q', 'chieff', 'z']
file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
gw_path = '/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/gw_events'
observations, obsdata, p_theta, events = gw_obs.generate_observations(params, gw_path, \
                                            100, 'posteriors', None)

model_names, flow = read_models.get_models(file_path, [channel_label], params, use_flows=True, device='cpu', no_bins=[5], use_unityweights=False)
_, KDE = read_models.get_models(file_path, [channel_label], params, use_flows=False, device='cpu', use_unityweights=False)

#inputs: x, data, pop_models, submodels_dict, channels, use_flows
hyperparams = list(set([x.split('/', 1)[1] for x in model_names]))
Nhyper = np.max([len(x.split('/')) for x in hyperparams])
channels = sorted(list(set([x.split('/')[0] for x in model_names])))

# construct dict that relates submodels to their index number
submodels_dict = {} #dummy index dict keys:0,1,2,3, items: particular models
ctr=0 #associates with either chi_b or alpha (0 or 1)
while ctr < Nhyper:
    submodels_dict[ctr] = {}
    hyper_set = sorted(list(set([x.split('/')[ctr] for x in hyperparams])))
    for idx, model in enumerate(hyper_set): #idx associates with 0,1,2,3,(4) keys
        submodels_dict[ctr][idx] = model
    ctr += 1

100%|██████████| 20/20 [00:08<00:00,  2.34it/s]


In [4]:
flow_path="/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/rns/Flows_150124_extralong/flow_models/"
flow[channel_label].load_model(flow_path, channel_label)

In [5]:
np.random.seed(92)

In [14]:
no_samples =100000
flow_KDE_KL = np.zeros((4,5))

for chi_b_id, xb in enumerate(chi_b):
    for alpha_id, a in enumerate(alpha_CE):

        flow_samples_stack = np.exp(flow[channel_label](obsdata, np.array([chi_b_id,alpha_id]), 9909, p_theta))

        kde_samples = KDE[channel_label][submodels_dict[0][0]][submodels_dict[1][0]](obsdata, 9909, p_theta)

        model_samples_idx = np.random.choice(np.shape(models_dict[(chi_b_id,alpha_id)])[0], no_samples, \
            p=weights_dict[(chi_b_id,alpha_id)]/np.sum(weights_dict[(chi_b_id,alpha_id)]))
        model_samples = np.array(models_dict[(chi_b_id,alpha_id)])[model_samples_idx]

        flow_KDE_KL[chi_b_id,alpha_id] = entropy(flow_samples_stack, kde_samples)
        print(flow_KDE_KL[chi_b_id,alpha_id])

        

0.3134718516310004
1.5754119777909987


KeyboardInterrupt: 